In [8]:
import random
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit import transpile

In [9]:
n_qubits = 10
eve_fraction = 0.3  # 30% dos qubits serão interceptados por Eve
noise_p = 0.05      # 5% de probabilidade de erro no canal

In [10]:
alice_bits = [random.randint(0, 1) for _ in range(n_qubits)]
alice_bases = [random.choice(['Z', 'X']) for _ in range(n_qubits)]

In [11]:
bob_bases = [random.choice(['Z', 'X']) for _ in range(n_qubits)]
bob_results = []

In [12]:
simulator = Aer.get_backend('qasm_simulator')

for i in range(n_qubits):
    qc = QuantumCircuit(1, 1)
    
    # Preparação do estado quântico baseado nos bits e bases de Alice
    if alice_bases[i] == 'Z':
        if alice_bits[i] == 1:
            qc.x(0)
    else:
        if alice_bits[i] == 0:
            qc.h(0)
        else:
            qc.x(0)
            qc.h(0)
    
    # Simulação da interceptação por Eve
    if random.random() < eve_fraction:
        eve_base = random.choice(['Z', 'X'])
        
        if eve_base == 'X':
            qc.h(0)
        qc.measure(0, 0)
        
        circuit_eve = transpile(qc, simulator)
        job_eve = simulator.run(circuit_eve, shots=1)
        counts_eve = job_eve.result().get_counts()
        eve_result = int(list(counts_eve.keys())[0])
        
        qc = QuantumCircuit(1, 1)
        if eve_base == 'Z':
            if eve_result == 1:
                qc.x(0)
        else:
            if eve_result == 0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
    
    # Aplicar ruído no canal
    if random.random() < noise_p:
        qc.x(0)
    
    # Medição do qubit baseado nas bases escolhidas por Bob
    if bob_bases[i] == 'X':
        qc.h(0)
    
    qc.measure(0, 0)
    
    circuit = transpile(qc, simulator)
    job = simulator.run(circuit, shots=1)
    counts = job.result().get_counts()
    measured_bit = int(list(counts.keys())[0])
    bob_results.append(measured_bit)

In [13]:
sifted_key_alice = []
sifted_key_bob = []

for i, (a_bit, a_base, b_base, b_result) in enumerate(zip(alice_bits, alice_bases, bob_bases, bob_results)):
    if a_base == b_base:
        sifted_key_alice.append(a_bit)
        sifted_key_bob.append(b_result)

In [14]:
if len(sifted_key_alice) > 0:
    errors = sum(a != b for a, b in zip(sifted_key_alice, sifted_key_bob))
    qber = errors / len(sifted_key_alice)
    print(f"QBER: {qber:.2%}")
else:
    print("Nenhuma base coincidente!")

QBER: 0.00%
